In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error as mse
import os
import warnings
warnings.simplefilter(action='ignore')

In [2]:
data =  pd.read_csv('../../data/province-month.csv')
province_codes = [10, 41, 50, 70, 90]

which_model = 'default_prophet'
prov = 50
year = 2011
month = 3

folder_name = '../../output/monthly_forecasts/' + which_model + '/'
prov_folder = folder_name + 'prov_' + str(prov) + '_monthly/'
file_name = prov_folder + 'prov_' + str(prov) + '_' + str(year) + '_' + str(month) + '_monthly.csv'
forecast_df = pd.read_csv(file_name)

forecast_cases = np.array(forecast_df['yhat'])

date = np.array(forecast_df['ds'])

In [3]:
data.head()
prov_data = data.loc[data['province'] == prov]
blah = prov_data.loc[prov_data['date_sick_year'] == 2016]
print(blah)

       date_sick_year  month  province  cases
47498            2016      1        50    7.0
47570            2016      2        50    4.0
47644            2016      3        50    1.0
47713            2016      4        50    4.0
47784            2016      5        50   12.0
47853            2016      6        50   16.0
47928            2016      7        50  167.0
48004            2016      8        50  284.0
48078            2016      9        50  165.0
48151            2016     10        50   84.0
48223            2016     11        50   48.0
48290            2016     12        50   14.0


In [4]:
forecast_cases
date

array(['2011-02-28', '2011-03-31', '2011-04-30', '2011-05-31',
       '2011-06-30', '2011-07-31', '2011-08-31', '2011-09-30',
       '2011-10-31', '2011-11-30', '2011-12-31', '2012-01-31'], dtype=object)

In [5]:
# years  2008 2008 2008 2008 2008 2008 2008 2008 2008 2008 2009 2009
# months 3    4    5    6    7    8    9    10   11   12   1    2

months_first = np.arange(month, 13)
months_last = np.arange(1, month)

month_list = np.concatenate([months_first, months_last])

year_list = []
for i in month_list:
    if i >= month:
        year_list.append(year)
    else: 
        year_list.append(year + 1)

print(len(year_list))

12


In [7]:
new_year_list = [year if i >=month else year + 1 for i in month_list]
print(new_year_list)

[2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2009, 2009]


In [18]:

true_cases = []
for i in range(0, 12):
    year = year_list[i]
    month = month_list[i]
    cases_df = float(prov_data['cases'].loc[(prov_data['date_sick_year'] == year) &  (prov_data['month'] == month)])
    true_cases.append(cases_df)

true_cases = np.array(true_cases)

In [19]:
true_cases

array([  501.,   561.,   595.,   900.,  1054.,  1228.,   508.,  1355.,
         914.,   680.,   354.,   225.])

In [14]:
cases_df = prov_data['cases'].loc[(prov_data['date_sick_year'] == year) &  (prov_data['month'] == month)]
cases_df.dtype

dtype('float64')

In [69]:
print(forecast_cases)
print(true_cases)

errors = abs(true_cases - forecast_cases)
print()
print(errors)

[ 323.4264863   227.43773184  571.55992707  743.01418805  675.58282072
  624.35504617  672.81948518  670.6813715   710.2417155   583.83672762
  496.29922113  354.60581714]
[  501.   561.   595.   900.  1054.  1228.   508.  1355.   914.   680.
   354.   225.]

[ 177.5735137   333.56226816   23.44007293  156.98581195  378.41717928
  603.64495383  164.81948518  684.3186285   203.7582845    96.16327238
  142.29922113  129.60581714]


In [75]:
error_dict = {'year': year_list, 'month': month_list, 'error': errors}
error_df = pd.DataFrame(error_dict)

error_df

,error,month,year
0,177.573514,3,2008
1,333.562268,4,2008
2,23.440073,5,2008
3,156.985812,6,2008
4,378.417179,7,2008
5,603.644954,8,2008
6,164.819485,9,2008
7,684.318629,10,2008
8,203.758285,11,2008
9,96.163272,12,2008


In [52]:
prov_data.loc[prov_data['date_sick_year'] == 2009]

,date_sick_year,month,province,cases
41358,2009,1,10,354.0
41425,2009,2,10,225.0
41495,2009,3,10,219.0
41568,2009,4,10,208.0
41642,2009,5,10,287.0
41718,2009,6,10,584.0
41793,2009,7,10,513.0
41869,2009,8,10,622.0
41944,2009,9,10,548.0
42019,2009,10,10,644.0


In [7]:
data.head()

,date_sick_year,month,province,cases
0,1968,1,10,55.0
1,1968,1,11,3.0
2,1968,1,12,4.0
3,1968,1,13,0.0
4,1968,1,14,0.0
